# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [3]:
!cat talks.csv

title;type;url_slug;venue;date;location;description
Dynamical phase transition in assemblies of chemotactic cells;Invited Talk;inv-talk-1;DPG Spring Meeting, Focus Session on Stochastic processes in biology;2019-04-04;Regensburg Universität,  Regensburg, Germany;
Fluid pumping and flexoelectricity can drive lumen nucleation in cell spheroids;Invited Talk;inv-talk-2;"Labex ""Who am I?"" 2022 annual meeting";2022-12-12;ENSA, Paris, France;
Genuine (response-field) Langevin equations for reaction-diffusion processes ;Contributed talk;talk-1;Journées de Physique Statistique;2016-01-28;ESPCI, Paris, France;
Langevin equations for reaction-diffusion processes;Contributed talk;talk-2;DPG Spring Meeting;2018-03-12;Technische Universität Berlin, Berlin, Germany;
Frequency regulators for the nonperturbative renormalization group in nonequilibrium systems;Contributed talk;talk-3;Exact Renormalization Group (ERG2018);2018-07-10;Sorbonne Université, Paris, France;
Fluid pumping can drive lumen nucl

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [9]:
# conv = {'publi_id': lambda x: str(x), 'page': lambda x: str(x), 'arxiv_id': lambda x: str(x)}
conv = {}
talks = pd.read_csv("talks.csv", sep=";", header=0, converters=conv)#, engine ='python')
talks

,title,type,url_slug,venue,date,location,description
0,Dynamical phase transition in assemblies of ch...,Invited Talk,inv-talk-1,"DPG Spring Meeting, Focus Session on Stochasti...",2019-04-04,"Regensburg Universität, Regensburg, Germany",NaN
1,Fluid pumping and flexoelectricity can drive l...,Invited Talk,inv-talk-2,"Labex ""Who am I?"" 2022 annual meeting",2022-12-12,"ENSA, Paris, France",NaN
2,Genuine (response-field) Langevin equations fo...,Contributed talk,talk-1,Journées de Physique Statistique,2016-01-28,"ESPCI, Paris, France",NaN
3,Langevin equations for reaction-diffusion proc...,Contributed talk,talk-2,DPG Spring Meeting,2018-03-12,"Technische Universität Berlin, Berlin, Germany",NaN
4,Frequency regulators for the nonperturbative r...,Contributed talk,talk-3,Exact Renormalization Group (ERG2018),2018-07-10,"Sorbonne Université, Paris, France",NaN
5,Fluid pumping can drive lumen nucleation and t...,Contributed talk,talk-4,Circle Meeting,2019-03-27,"Saarland Universität, Saarbrücken, Germany",NaN
6,Hydraulic and electric control of cell spheroids,Contributed talk,talk-5,Virtual APS March Meeting,2021-03-16,online conference,NaN
7,Hydraulic and electric control of cell spheroids,Contributed talk,talk-6,Virtual DPG Spring Meeting,2021-03-23,online conference,NaN
8,Nonlinear rheology of cellular networks,Contributed talk,talk-7,Physical Biology Circle Meeting,2021-09-15,Max Planck Institute for the Physics of Comple...,NaN
9,Hydraulic and electric control of a cell spheroid,Contributed talk,talk-8,New frontiers in liquid matter,2022-07-07,"Sorbonne Université, Paris, France",NaN


In [6]:
# talks = pd.read_csv("talks.tsv", sep="\t", header=0)
# talks

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [5]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [10]:
loc_dict = {}

for row, item in talks.iterrows():
    print(item.type)

Invited Talk
Invited Talk
Contributed talk
Contributed talk
Contributed talk
Contributed talk
Contributed talk
Contributed talk
Contributed talk
Contributed talk
Seminar
Seminar
Seminar
Seminar
Seminar
Seminar
Seminar
Seminar
Seminar
Seminar
Seminar
Seminar
Seminar
Poster
Poster
Poster
Poster
Poster
Poster
Poster
Poster
Poster
Poster


In [6]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
#     if len(str(item.talk_url)) > 3:
#         md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [7]:
!ls ../_talks

2012-05-21-poster-1.md   2019-10-21-poster-7.md   2021-06-07-poster-9.md
2016-01-28-talk-1.md     2020-01-07-seminar-2.md  2021-07-26-poster-10.md
2016-08-01-poster-2.md   2020-01-09-seminar-3.md  2021-09-15-talk-7.md
2017-02-09-poster-3.md   2020-01-15-seminar-4.md  2021-09-15-talk-8.md
2017-03-30-seminar-1.md  2020-03-31-seminar-5.md  2021-12-13-seminar-9.md
2018-03-12-talk-2.md     2020-10-14-seminar-6.md  2022-01-28-seminar-10.md
2018-07-10-talk-3.md     2020-10-21-poster-8.md   2022-02-16-seminar-11.md
2018-08-01-poster-4.md   2020-11-02-seminar-7.md  2022-02-22-seminar-12.md
2019-03-27-talk-4.md     2020-11-10-seminar-8.md  2022-07-07-talk-8.md
2019-04-04-inv-talk-1.md 2021-03-16-talk-5.md     2022-10-10-seminar-13.md
2019-04-04-talk-5.md     2021-03-16-talk-6.md     2022-12-12-inv-talk-2.md
2019-05-01-poster-5.md   2021-03-23-talk-6.md     talkmap.ipynb
2019-07-03-poster-6.md   2021-03-23-talk-7.md


In [11]:
!cat ../_talks/2016-01-28-talk-1.md

---
title: "Genuine (response-field) Langevin equations for reaction-diffusion processes "
collection: talks
type: "Contributed talk"
permalink: /talks/2016-01-28-talk-1
venue: "Journées de Physique Statistique"
date: 2016-01-28
location: "ESPCI, Paris, France"
---


In [12]:
!cat ../_talks/2019-04-04-inv-talk-1.md

---
title: "Dynamical phase transition in assemblies of chemotactic cells"
collection: talks
type: "Invited Talk"
permalink: /talks/2019-04-04-inv-talk-1
venue: "DPG Spring Meeting, Focus Session on Stochastic processes in biology"
date: 2019-04-04
location: "Regensburg Universität,  Regensburg, Germany"
---
